In [ ]:
import tensorflow as tf # Models ran in venv python 3.9.16 with GPU computing support
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import wfdb
import ast
import ecg_plot
print(tf.config.list_physical_devices()) # Verify you have a GPU available. Not required
print(tf.test.is_built_with_cuda())
print(tf.test.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None))

In [ ]:
sr=500

In [ ]:
import pandas as pd
import numpy as np
import wfdb
import ast
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, BatchNormalization, Activation, Dropout
from keras.utils import to_categorical
sr =100

with tf.device('/CPU:0'):
    ### Loading raw data into mutable Datframes
    ptb = pd.read_csv('../data/ptbxl_database.csv')
    def load_raw_data(df, sampling_rate, path):
        if(sampling_rate == 100):
            data = [wfdb.rdsamp(path+f) for f in df.filename_lr]
        else:
            data = [wfdb.rdsamp(path+f) for f in df.filename_hr]
        data = np.array([signal for signal, meta in data])
        return data
    
    # load and convert annotation data
    Y = pd.read_csv('../data/ptbxl_database.csv', index_col='ecg_id')
    Y.scp_codes = Y.scp_codes.apply(lambda x: ast.literal_eval(x))

    # Load raw signal data
    X = load_raw_data(Y, sr, '../data/')

    # Load scp_statements.csv for diagnostic aggregation
    agg_df = pd.read_csv('../data/scp_statements.csv', index_col=0)
    agg_df = agg_df[agg_df.diagnostic == 1]

    def aggregate_diagnostic(y_dic):
        tmp = []
        for key in y_dic.keys():
            if key in agg_df.index:
                tmp.append(agg_df.loc[key].diagnostic_class)
        return list(set(tmp))


    # Apply diagnostic superclass
    Y['diagnostic_superclass'] = Y.scp_codes.apply(aggregate_diagnostic)


In [ ]:
import numpy as np
from scipy import signal
with tf.device('/CPU:0'):
    # Define the filter parameters
    fs = 100  # Sampling frequency (Hz)
    lowcut = 0.5  # Lower cutoff frequency (Hz)
    highcut = 40.0  # Higher cutoff frequency (Hz)
    filter_order = 4  # Filter order

    def bandpass(X, fs, lowcut, highcut, filter_order):
        # Apply bandpass filter to each channel
        filtered_data = np.zeros_like(X)
        for i in range(X.shape[2]):
            for j in range(X.shape[0]):
                b, a = signal.butter(filter_order, [lowcut, highcut], fs=fs, btype='band', output='ba')
                filtered_data[j, :, i] = signal.filtfilt(b, a, X[j, :, i])

        # Print the shape of the filtered data
        return filtered_data

In [ ]:
with tf.device('/CPU:0'):
    def normalize(X, shape = False):
        # Compute mean and standard deviation along axis 1 and 2
        X_mean = np.mean(X)
        X_std = np.std(X)

        if(shape):
            print(X.shape)
        # Normalize data by subtracting mean and dividing by standard deviation
        return ((X - X_mean) / X_std)


    bld = 0.5
    def baseline_drift(X, baseline, range = 0):
        bld_range = (baseline-range, baseline+range)
        random_shifts = np.random.uniform(bld_range[0], bld_range[1], size = X.shape)
        return X + random_shifts

In [ ]:
with tf.device('/GPU:0'):
    def da_apply(X, functions, shape = False):
        X_final = X.copy()

        # Applys DA Augments in specified order
        for func_dict in functions:
            func = func_dict['func']
            if(func_dict['params'] == None):
                X_final = func(X_final)     
            else:
                params = func_dict['params']
                X_final = func(X_final, *params)  

        if(shape == True):
            print(X_final.shape)

        return X_final

    #func_dict = [{'func': normalize, 'params': [None]}]

    #func_dict = [{'func': baseline_drift, 'params': [0,0.05]}]

    func_dict =[{'func': normalize, 'params': [None]},
                {'func': baseline_drift, 'params': [0,0.075]}]


    # func_dict =[{'func': bandpass, 'params': [sr, 0.5, 15, 3]}, # 100, 0.5, 10, 3. Change the sampling rate as necessary
    #             {'func': normalize, 'params': [None]},  # Normalizing across the entire data set instead of by lead
    #             {'func': baseline_drift, 'params': [0,0.075]}]  # I found that simulating a 0.075 drift works best

    X_final = da_apply(X, func_dict) # If you want to experiment with data shape more, use a smaller data size. Subset X

In [ ]:

with tf.device('/GPU:0'):
    # Split data into train and test
    test_fold =10
    val_fold = 9

    def tvt_split(X, Y, val_fold, test_fold, shape = False):
        X_train = X[(Y.strat_fold != test_fold) & (Y.strat_fold != val_fold)]
        y_train = Y[(Y.strat_fold != test_fold) & (Y.strat_fold != val_fold)].age

        X_val = X[Y.strat_fold == val_fold]
        y_val = Y[Y.strat_fold == val_fold].age

        X_test = X[(Y.strat_fold == test_fold)]
        y_test = Y[Y.strat_fold == test_fold].age

        y_train = pd.get_dummies(y_train)
        y_val = pd.get_dummies(y_val)
        y_test = pd.get_dummies(y_test)

        y_train = y_train.idxmax(axis = 1).to_numpy()
        y_val = y_val.idxmax(axis = 1).to_numpy()
        y_test = y_test.idxmax(axis = 1).to_numpy()

        rX_train = X_train[(y_train < 89) & (y_train >= 18)] # Additional filtering of patients older than 89 and younger than 18
        ry_train = y_train[(y_train < 89) & (y_train >= 18)]

        rX_val = X_val[(y_val < 89) & (y_val >= 18)]
        ry_val = y_val[(y_val < 89) & (y_val >= 18)]

        rX_test = X_test[(y_test < 89) & (y_test >= 18)]
        ry_test = y_test[(y_test < 89) & (y_test >= 18)]

        if(shape == True):
            print((X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape))

        return rX_train, ry_train, rX_val, ry_val, rX_test, ry_test

    X_train, y_train, X_val, y_val, X_test, y_test = tvt_split(X_final, Y, val_fold, test_fold, True)

In [83]:

def extract_avl(X_data):
    num_rows = len(X_data)
    
    X_result = np.zeros((num_rows, 1000))

    for i, outer in enumerate(X_data):
        fv = np.zeros(1000)
        fv = [inner_list[3] for inner_list in outer]
        X_result[i, :len(fv)] = fv  

    return X_result

def extract_avr(X_data):
    num_rows = len(X_data)
    
    X_result = np.zeros((num_rows, 1000))

    for i, outer in enumerate(X_data):
        fv = np.zeros(1000)
        fv = [inner_list[4] for inner_list in outer]
        X_result[i, :len(fv)] = fv  

    return X_result
X_tr = extract_avl(X_train)

X_va = extract_avl(X_val)

X_te = extract_avl(X_test)


In [89]:
X_tra = extract_avr(X_train)
X_val = extract_avr(X_val)
X_te = extract_avr(X_test)

In [87]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

In [93]:
model = LinearRegression()
model.fit(X_tr, y_train)

y_pred = model.predict(X_va)

mae = mean_absolute_error(y_val, y_pred)
print("Mean Absolute Error on Validation Set:", mae)

y_test_pred = model.predict(X_te)
ma =mean_absolute_error(y_test,y_test_pred)
ma

Mean Absolute Error on Validation Set: 14.003559453859296


14.249230757931887